In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

x = np.linspace(0, 9, 10)
y = np.linspace(10, 19, 10)
lon, lat = np.meshgrid(x, y)
times = pd.date_range("2020-01-01", end = '2020-12-31', freq = '6D')
n = len(times)
data = np.random.randn(10,10, n)

ros = np.resize([1, 24], (n))
platforms = np.resize(['S1A', 'S1B', 'S1B', 'S1A'], (n))
direction = np.resize(['ascending', 'descending'], (n))

ds = xr.Dataset(data_vars= dict(
    data = (["x", "y", "time"], data)
        ),
    coords = dict(
                        lon = (["x", "y"], lon),
                        lat = (["x", "y"], lat),
                        time = times,
                        relative_orbit = (["time"], ros),
                        platform = (["time"], platforms),
                        flight_dir = (["time"], direction)) 
        
    )

import sys
sys.path.append('/Users/zachkeskinen/Documents/spicy-snow/')
from spicy_snow.processing.s1_preprocessing import subset_s1_images, merge_s1_subsets

import pickle
with open('/Users/zachkeskinen/Documents/spicy-snow/tests/test_data/search_result', 'rb') as f:
    ds = pickle.load(f)
# ds = ds.isel(time = slice(0, 3))

In [ ]:
import hyp3_sdk as sdk

hyp3 = sdk.HyP3()

# gather granules to submit to the hyp3 pipeline
granules = 'S1B_IW_GRDH_1SDV_20200201T013528_20200201T013553_020069_025FB3_6D5E'

# create a new hyp3 batch to hold submitted jobs
rtc_jobs = sdk.Batch()

rtc_jobs += hyp3.submit_rtc_job(granules, name = 'db test', include_inc_map = True,\
    scale = 'decibel', dem_matching = False, resolution = 30)

# warn user this may take a few hours for big jobs
print(f'Watching {len(rtc_jobs)} jobs. This may take a while...')

# have hyp3 watch and update progress bar every 60 seconds
hyp3.watch(rtc_jobs)

# refresh jobs list with successes and failures
rtc_jobs = hyp3.refresh(rtc_jobs)

In [ ]:
from spicy_snow.utils.download import url_download
import rioxarray as rxa

u = rtc_jobs[0].files[0]['url']

# create dictionary to hold cloud url from .zip url
# this lets us download only VV, VH, inc without getting other data from zip
urls = {}
urls[f'{granules}_VV'] = u.replace('.zip', '_VV.tif')


# download url to a tif file
url_download(u.replace('.zip', '_VV.tif'), './vv_db.tif', verbose = False)

# open image in xarray
img = rxa.open_rasterio('./vv_db.tif', masked = True)